# Downloading Libraries

In [1]:
!pip install python-gdcm
!pip install -U pylibjpeg[all]
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.3/969.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 697.9 kB/s eta 0:00:00


# Importing and Installing Libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pathlib
import random
import timm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
import albumentations.pytorch

from fastai.vision.all import *
from fastai.data.core import *
from sklearn.model_selection import StratifiedShuffleSplit
# Setting Seed for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)  
torch.cuda.manual_seed(seed)  
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparamters

In [3]:
# timm.list_models()

In [4]:
model_name = 'resnet50' # Change the model here to resnet 50 or efficient net
epochs = 6
batch_size = 8
number_of_slices = 160 # Mean of, mean and median of no of slices
input_shape = (224, 224)
resize_size = 256
num_of_hidden = 2
hidden_dimension = [256, 64]
output_categories = 8
loss_weights = {'-ve': torch.tensor([1., 1., 1., 1., 1., 1., 1., 7.]),
                '+ve': torch.tensor([2., 2., 2., 2., 2., 2., 2., 14.])}
criterion = nn.BCEWithLogitsLoss(reduction = 'none')
train_size = 0.8
val_size = 0.2
lr_freeze = 0.0001
lr_unfreeze = slice(3e-07, 3e-06)
save_path = '/kaggle/working/'

# Helper Function

In [5]:
def loss(y_true, y_pred):
    losses = nn.BCEWithLogitsLoss()
    loss = losses(y_true,y_pred)
    weights  = y_true*loss_weights['+ve'].to(device) + (1-y_true)*loss_weights['-ve'].to(device)
    loss = (loss * weights).sum(axis=1)
    loss = loss.mean()
    loss = (loss / weights.sum(axis=1)).sum()
    return loss

def metric(y_true, y_pred):
    losses = nn.BCEWithLogitsLoss()
    loss = losses(y_true,y_pred)
    weights  = y_true*loss_weights['+ve'].to(device) + (1-y_true)*loss_weights['-ve'].to(device)
    loss = (loss * weights).sum(axis=1)
    loss = loss.mean()
    loss = (loss / weights.sum(axis=1)).sum()
    return loss

# Loading Data

In [6]:
base_path = pathlib.Path('/kaggle/input/rsna-2022-cervical-spine-fracture-detection')
df = pd.read_csv(base_path/'train.csv')
df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [7]:
df = df[df['StudyInstanceUID'] != '1.2.826.0.1.3680043.20574'].copy()
print(len(df))

2018


In [8]:
df['path'] = list(map(lambda x: base_path/'train_images'/x, df['StudyInstanceUID']))
df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,path
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.21561
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.12351
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.1363


In [9]:
# min_slices = 1500
# max_slices = 0
# mean_slices = 0
# count = 0
# slices_val = []
# for i in df['path']:
#     no_of_images = len(list(i.glob("*")))
#     count+=1
#     slices_val.append(no_of_images)
#     mean_slices += no_of_images
#     if min_slices > no_of_images:
#         min_slices = no_of_images
#     if no_of_images > max_slices:
#         max_slices = no_of_images
# print("Minimum Slices in an Image", min_slices)
# print("Maximum Slices in an Image", max_slices)
# print("Mean Slices in an Image", mean_slices / count)
# slices_val.sort()
# print("Median Slices in an Image", slices_val[(count + 1)//2])

In [10]:
strat = StratifiedShuffleSplit(n_splits=2, test_size = val_size/(train_size + val_size), 
                                random_state=seed)
for (train_idx, valid_idx) in strat.split(df.index, df['C3']):
    valid_data = df.iloc[valid_idx]
    train_data = df.iloc[train_idx]

In [11]:
len(train_data), len(valid_data)

(1614, 404)

In [12]:
train_data['patient_overall'].value_counts(normalize = True)

0    0.521066
1    0.478934
Name: patient_overall, dtype: float64

In [13]:
valid_data['patient_overall'].value_counts(normalize = True)

0    0.537129
1    0.462871
Name: patient_overall, dtype: float64

In [14]:
class CervicalDataset(Dataset):
    def __init__(self, df, no_of_slice, prob = 0.5, test = False, tta = False):
        '''
        '''
        self.df = df
        self.test = test
        self.tta = tta
        self.prob = prob
        self.no_of_slice = no_of_slice
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df['path'].iloc[idx]
        images_path = list(data.glob("*"))
        no_of_images = len(images_path)
        transforms_list = [A.LongestMaxSize(max_size=resize_size, interpolation=1),
                           A.PadIfNeeded(min_height=input_shape[0], min_width=input_shape[1],
                                         border_mode=0, value=(0,0,0))]
        if not(self.test):
            value = np.random.uniform()
            if value >= self.prob: 
                transforms_list.append(A.HorizontalFlip(always_apply = True))
                
            value = np.random.uniform()
            if value >= self.prob:
                transforms_list.append(A.ShiftScaleRotate(shift_limit = 0.15,
                                                          border_mode = 2, always_apply = True))
            
            # Other Transformation to add: Random Brightness, Contrast, Clahe, Scale Intensity 
                
            transforms_list.append(A.CenterCrop(height = input_shape[0], width = input_shape[1]))
            transforms = A.Compose(transforms_list)
        elif self.test and self.tta:
            pass
        if self.test:
            transforms_list.append(A.CenterCrop(height = input_shape[0], width = input_shape[1]))
            transforms = A.Compose(transforms_list)
            
        imgs = []
        for i in range(1, len(images_path)+1):
            path = images_path[0].parent/f"{i}.dcm"
            
            try:
                data = pydicom.dcmread(path)
#                 data.PhotometricInterpretation = 'YBR_FULL'
                img_data = apply_voi_lut(data.pixel_array, data)
                img_data = img_data - np.min(img_data)
                if np.max(img_data) != 0:
                    img_data = img_data / np.max(img_data)
    #             data = (data * 255).astype(np.uint8)
                album = transforms(image = img_data)
                img_data = album['image']
            except FileNotFoundError:
                continue
            imgs.append(img_data)
            if len(imgs) > self.no_of_slice:
                break
            
        if len(imgs) > self.no_of_slice:
            imgs = imgs[:self.no_of_slice]
        
        if len(imgs) < self.no_of_slice:
            imgs.extend([np.zeros((input_shape[0], input_shape[1]))
                         for i in range(self.no_of_slice - len(imgs))])

        imgs = np.array(imgs)
        imgs = torch.from_numpy(imgs).float()
        labels = torch.as_tensor(self.df.iloc[idx].iloc[1:-1]).float()
        return imgs, labels

In [15]:
def get_data(batch_size):
    train_dataset = CervicalDataset(train_data, no_of_slice = number_of_slices)
    valid_dataset = CervicalDataset(valid_data, no_of_slice = number_of_slices, test = True)
    dls = DataLoaders.from_dsets(train_dataset, valid_dataset, bs = batch_size)
    return dls

In [16]:
# train_dataset = CervicalDataset(train_data, no_of_slice = number_of_slices)
# trainloader = DataLoader(train_dataset, batch_size = 16)

In [17]:
# imgs, lbls = next(iter(trainloader))
# print(imgs.shape, lbls.shape)

In [18]:
# # Visualize
# image = imgs[3]
# print(image.shape) # No_of_slice x height x width
# fig, axes = plt.subplots(nrows = 1, ncols = 4, figsize = (15,10))
# axes[0].imshow(image.numpy().mean(axis = 0), cmap = 'gray')
# axes[0].axis('off')
# axes[1].imshow(image.numpy()[:, :, image.shape[2]//2], cmap = 'gray')
# axes[1].axis('off')
# axes[2].imshow(image.numpy()[:, 128, :], cmap = 'gray')
# axes[2].axis('off')
# axes[3].imshow(image.numpy()[6, :, :], cmap = 'gray')
# axes[3].axis('off');

# Model Building

In [19]:
# If using Efficient net use base.classifier instead of base.fc otherwise make no changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
        

class Network(nn.Module):
    def __init__(self, base, input_channels, number_of_hidden, hidden, output_categories, freeze_layer):
        super(Network, self).__init__()
        if (number_of_hidden != len(hidden)):
            raise "Number of Hidden layer and length of hidden dim must be same"
            
        
        hidden_dim = hidden[:]
        hidden_dim.insert(0, base.fc.in_features)
        
        
        self.p = 0.5
        
        self.cnn_head = nn.Sequential(nn.Conv2d(input_channels, 128, kernel_size = 5, stride = 1, 
                                                padding = 2, padding_mode = 'reflect'),
                                      nn.ReLU(),
                                      nn.Conv2d(128, 3, kernel_size = 5, stride = 1,
                                                padding = 2,  padding_mode = 'reflect'),
                                      nn.ReLU())
        base.fc = Identity()
        self.network = self.__freeze_layer(base, freeze_layer)
        self.classification = self.__fully_connected(number_of_hidden, hidden_dim, output_categories)
        self.__initialise_weights()

    def __initialise_weights(self):
        for m in self.classification:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
              
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        for m in self.cnn_head:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
              
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                

    def __freeze_layer(self, base, freeze_layer):
        cnt = 0
        for child in base.children():
            cnt+=1
            if cnt > freeze_layer:
                break

            for param in child.parameters():
                param.requires_grad = False
        return base


    def __fully_connected(self, number_of_hidden, hidden_dim, output_categories):
        layers = []
        layers.append(nn.Dropout(self.p))
        for i in range(number_of_hidden):
            layers.append(nn.Linear(hidden_dim[i], hidden_dim[i+1]))
            layers.append(nn.GELU())
            layers.append(nn.BatchNorm1d(hidden_dim[i+1]))
            if i != (number_of_hidden-1):
                layers.append(nn.Dropout(self.p))

        layers.append(nn.Linear(hidden_dim[-1], output_categories))
    
        return nn.Sequential(*layers)
        
    def forward(self,x):
        x = self.cnn_head(x)
        x = self.network(x)
        out = self.classification(x)
        return out

In [20]:
def get_learner(model_name, batch_size, unfreeze, metric, save_path):
    dls = get_data(batch_size).to(device)
    if unfreeze:
        network = timm.create_model(model_name, pretrained = True)
        model = Network(network, number_of_slices, num_of_hidden, hidden_dimension, output_categories, 0)
    else:
        network = timm.create_model(model_name, pretrained = True)
        model = Network(network, number_of_slices, num_of_hidden, hidden_dimension, output_categories, 8)
        
    model = model.to(device)
    learn = Learner(dls, model, loss_func=loss, metrics=metric, 
                    model_dir = save_path).to_fp16()
    return learn

In [21]:
learn = get_learner(model_name, batch_size, True, metric, save_path)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [22]:
# learn.lr_find()

In [23]:
# saved_file = '/kaggle/input/weights-trained/resnet_50'
# learn.load(saved_file)

/opt/conda/lib/python3.7/site-packages/fastai/learner.py:58: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [25]:
learn.fit_flat_cos(epochs, lr_unfreeze, wd = 0.1,
                cbs=[SaveModelCallback(monitor = 'metric', comp = np.less, fname = 'resnet_50_unfreeze')]) 

epoch,train_loss,valid_loss,metric,time
0,6.221916,9.412472,9.412472,1:24:07
1,6.136239,8.041072,8.041072,1:17:24
2,9.317728,7.972219,7.972219,1:16:03
3,10.127910,7.430823,7.430823,1:01:30


Better model found at epoch 0 with metric value: 9.412471771240234.
Better model found at epoch 1 with metric value: 8.041071891784668.
Better model found at epoch 2 with metric value: 7.9722185134887695.
Better model found at epoch 3 with metric value: 7.430822849273682.
